In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
import numpy as np

plt.style.use('paper.mplstyle')

In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

# table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue.fits' %root_dir)
# table_assoc = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations_wprops.fits' %root_dir)
table_muse = QTable.read('%s/data_misc/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits' %root_dir)

table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_fixediso_10pc.fits' %root_dir)
table_assoc = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations_fixediso_10pc.fits' %root_dir)

table = table[(table['flag_star_MUSE']!=1) & (table['flag_edge_MUSE']!=1) & (table["flag_edge_hst"]!=1)] # remove stars and edge objects
table_assoc = table_assoc[(table_assoc['flag_star_MUSE']!=1) & (table_assoc['flag_edge_MUSE']!=1) & (table_assoc["flag_edge_hst"]!=1)] # remove stars and edge objects
table_muse = table_muse[(table_muse['flag_star']!=1) & (table_muse['flag_edge']!=1)] # remove stars and edge objects

table = table[table['HII_class_v3_MUSE']==1]
table_assoc = table_assoc[table_assoc['HII_class_v3_MUSE']==1]
table_muse = table_muse[table_muse['HII_class_v3']==1]

table_assoc = table_assoc['gal_name', 'region_ID', 'region_ID_association_nuv_32pc', 'age_association_nuv_32pc', 'mass_association_nuv_32pc']
table_assoc = table_assoc[np.isfinite(table_assoc['age_association_nuv_32pc'])]
table_assoc = table_assoc[table_assoc['mass_association_nuv_32pc']>1]

print(f'Table length post-masking (inc. HII regions only): {len(table)}')
print(f'Table length post-masking (inc. HII regions, and with clusters): {len(table_assoc)}')

Table length post-masking (inc. HII regions only): 5015
Table length post-masking (inc. HII regions, and with clusters): 3227


In [3]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]
columns = ['gal_name'] + [f'percentile_{p}' for p in percentiles]
data = []

for gal_name in np.unique(table['gal_name']):

    table_masked = table[table['gal_name'] == gal_name]
    table_muse_masked = table_muse[table_muse['gal_name'] == gal_name]

    gal_data = [gal_name.replace('NGC', 'NGC~').replace('IC', 'IC~')]

    p_age = np.nanpercentile(table_assoc[table_assoc['gal_name'] == gal_name]['age_association_nuv_32pc'], percentiles)
    p_mass = np.log10(np.nanpercentile(table_assoc[table_assoc['gal_name'] == gal_name]['mass_association_nuv_32pc'], percentiles))

    for values in [p_age, p_mass]:
        for value in values:
            gal_data.extend([value])
    
    data.append(gal_data)

In [4]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]

gal_data = ['All']

p_age = np.nanpercentile(table_assoc['age_association_nuv_32pc'], percentiles)
p_mass = np.log10(np.nanpercentile(table_assoc['mass_association_nuv_32pc'], percentiles))

for values in [p_age, p_mass]:
    for value in values:
        gal_data.extend([value])

data.append(gal_data)

In [5]:
# Create a new QTable with the calculated percentiles
names1 = ['gal_name']
names2 = [f'p{p}_age' for p in percentiles]
names3 = [f'p{p}_mass' for p in percentiles]

names = names1 + names2 + names3
new_table = QTable(rows=data, names=names)

new_table

gal_name,p10_age,p50_age,p90_age,p10_mass,p50_mass,p90_mass
str8,float64,float64,float64,float64,float64,float64
IC~5332,1.0,1.0,4.300000000000004,3.05176020665029,3.6242583649609927,4.207316872897091
NGC~0628,1.0,2.0,8.4,3.201597130023678,3.862453141062399,4.614040783194185
NGC~1087,1.0,3.0,8.0,3.46785332510919,4.113319714388521,4.914031806884155
NGC~1300,1.0,4.0,6.0,3.283923353143175,3.916380838983697,4.544268721689977
NGC~1365,1.0,3.0,7.5,3.6169863804359,4.370047172135086,5.364268821981902
NGC~1385,1.0,4.0,8.0,3.4965135559444844,4.1997659542979795,5.01644289559053
NGC~1433,1.0,1.0,6.0,3.106149384426681,4.022829187915416,4.572377332991849
NGC~1512,1.0,1.0,7.600000000000001,3.426453861226114,4.045721759372231,4.816252750020856
NGC~1566,1.0,2.0,7.000000000000001,3.5882175601587813,4.189927131744368,5.084785511848023


In [6]:
names = ['gal_name',
        'p10_age',
        'p50_age',
        'p90_age',
        'p10_mass',
        'p50_mass',
        'p90_mass'
        ]

formats={
        'gal_name': '%s',
        'p10_age': '%.1f',
        'p50_age': '%.1f',
        'p90_age': '%.1f',
        'p10_mass': '%.2f',
        'p50_mass': '%.2f',
        'p90_mass': '%.2f'
        }

new_table.write('./tabs/agemass_props.tex', formats=formats, names=names, overwrite=True)
# new_table.write('./tabs/lumrad_props.fits', overwrite=True)
# new_table.write('./tabs/lumrad_props.dat', format='ascii', overwrite=True)